# Submission of Calculation with Subsequent Results Query

In this demo, we will submit a calculation, check the status of the job and query the results after it is finished.

In [ ]:
from azure.quantum import Workspace
from azure.quantum.job import JobFailedWithResultsError

# insert connection string form Azure Portal Workspace Access Keys
connection_string = ""
workspace = Workspace.from_connection_string(connection_string)

In [ ]:
# To submit Accelerated DFT jobs, we will be using the microsoft.dft target in the workspace.
print("Verifying access to Accelerated DFT target.")
target = workspace.get_targets("microsoft.dft")
print("Verification complete.")

Now we submit the calculations to MADFT service.

The QCSchema input below is for a DFT gradient calculation on the molecule phenol (with the geometry given in Bohr)

In [ ]:
qcschema_input = {
    "driver": "gradient",
    "model": {
        "method": "m06-2x",
        "basis": "def2-svpd"
    },
    "schema_name": "qcschema_input",
    "schema_version": 1,
    "molecule": {
        "extras": {},
        "symbols": [
            "O",
            "C",
            "C",
            "C",
            "C",
            "C",
            "C",
            "H",
            "H",
            "H",
            "H",
            "H",
            "H"
        ],
        "geometry": [
            4.730542147965709,
            0.034826575331843086,
            0.07810088784463559,
            2.1361232242687977,
            0.017709001458524106,
            0.009088108672780787,
            0.7996954919209014,
            2.290483253979806,
            0.10106814673106823,
            -1.8298562750208616,
            2.2732950799384737,
            -0.04537958079912547,
            -3.1327572801516967,
            -0.00564083248182671,
            -0.28742004920350506,
            -1.790388872477789,
            -2.271959799458856,
            -0.38978844089184156,
            0.8394687277399734,
            -2.2656284043593296,
            -0.24392044354214196,
            5.279447115915874,
            -0.07938333158181043,
            1.8109098053069272,
            1.8583211818406624,
            4.051452964636673,
            0.2691141588512759,
            -2.8675310249318393,
            4.053900197762506,
            0.0241508699472927,
            -5.190440656400895,
            -0.014523603513912258,
            -0.4052054313284032,
            -2.796624853566738,
            -4.060585444078858,
            -0.5909607661605761,
            1.9285725820008635,
            -4.013248220398251,
            -0.3415529925897059
        ]
    }
}

In [ ]:
# Submit job:
job_name = 'phenol_gradient'
target.submit(input_data=[qcschema_input],name=job_name)

# Query Job Status and Retreive Results

We can retrieve information about a job through [Workspace.get_job](https://learn.microsoft.com/en-us/azure/quantum/optimization-job-management), and also query the results by filtering the job name with [Workspace.list_jobs](https://learn.microsoft.com/en-us/azure/quantum/optimization-job-management).

In [ ]:
# query the latest job that match the given name
job = workspace.list_jobs(name_match=job_name)[-1]

# refresh the job SAS for using the API
job.refresh()

# show the status of the job
print(f'Job: "{job_name}" is {job.details.status}')

In [ ]:
# read the results of the job
if job.details.status == 'Succeeded':
    results = job.get_results()
else:
    results = f'"{job_name}" is still {job.details.status}...'

# QCSchema Output
qcschema = results["results"][0]

# Results

The results of the calculation are stored in the QCSchema format dict.

For an energy calculation we can see the energy by simply looking at the key "return_result".
For gradient calculations, this key returns the force.

In [ ]:
# The energy can be accessed:
energy = qcschema['properties']['return_energy']

# Since the job driver was 'gradient' the gradient is also returned 
gradient = qcschema["return_result"]

# or alternatively the gradient can be accessed via:
#gradient = qcschema['properties']['scf_total_gradient']

print("Total Energy (Hartree): ", energy)
print("Gradient: ", gradient)

Other useful information is stored in the output dict, for example:

In [ ]:
print("Number of Basis Functions: ", qcschema["properties"]["calcinfo_nbasis"])
print("Total Energy (Hartree): ", qcschema["properties"]["return_energy"])
print("Nuclear Repulsion Energy (Hartree): ", qcschema["properties"]["nuclear_repulsion_energy"])
print("Total Calculation Time (s): ", qcschema["provenance"]["total_time_seconds"])

## Output to QCSchema json file

In [ ]:
import json
n = job_name + "_output"
with open(n+".json", "w") as fp:
    json.dump(qcschema, fp, indent=4)
